In [1]:
import pandas as pd
import os


In [2]:
# This cell with take the training data file and generate a new training set with key features only

temp = pd.read_csv(
    "D:\Projects\Hackathons\shell hacks\mitre\FIU_Phishing_Mitre_Dataset.csv", 
    na_values=['NA', '?'])
temp.dropna()

df =pd.DataFrame(columns = ['create_age(months)','expiry_age(months)','update_age(days)','lengthOfUrl','#ofDots','#of/','#of-','firstURL'])
df['create_age(months)'] = temp['create_age(months)']
df['expiry_age(months)'] = temp['expiry_age(months)']
df['update_age(days)'] = temp['update_age(days)']
df['lengthOfUrl'] = temp['URL'].apply(len)
df['#ofDots'] = temp['URL'].apply(lambda x: x.count('.'))
df['#of/'] = temp['URL'].apply(lambda x: x.count('/'))
df['#of-'] = temp['URL'].apply(lambda x: x.count('-'))
df['Label'] = temp['Label']

start = 'http://'
end = '/'
df['firstURL'] = temp['URL'].apply(lambda x: x[x.find(start)+len(start):x.rfind(end)])
df['firstURL'] = df['firstURL'].apply(len)

export_csv = df.to_csv(r'TrainingSet.csv',index=None,header=True)


In [3]:
# Importing for the traing model

from scipy.stats import zscore
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils import shuffle
from tensorflow.keras.regularizers import l2

In [4]:
# loading the new training set and getting it ready for the keras model

df_new = pd.read_csv(
    "D:/Projects/Hackathons/shell hacks/mitre/TrainingSet.csv",
    na_values=['NA','?'])

#df_new[df_new < 0] = 0
#df_new['create_age(months)'] = zscore(['create_age(months)'])
#df_new['expiry_age(months)'] = zscore(['expiry_age(months)'])
#df_new['update_age(days)'] = zscore(['update_age(days)'])
df_new = shuffle(df_new)


x_columns = df_new.columns.drop('Label')
x = df_new[x_columns].values
dummies = pd.get_dummies(df_new['Label']) # Classification
species = dummies.columns
y = dummies.values


x_train, x_test, y_train, y_test = train_test_split(    
    x, y, test_size=0.20, random_state=42)
df_new

,create_age(months),expiry_age(months),update_age(days),lengthOfUrl,#ofDots,#of/,#of-,firstURL,Label
1100,3,9,27,37,4,5,0,26,1
3918,91,5,59,47,4,6,0,36,1
3171,-1,-1,-1,48,7,5,0,27,1
3870,45,3,267,38,5,4,0,27,1
2453,111,9,124,58,5,4,2,47,1
...,...,...,...,...,...,...,...,...,...
4293,-1,-1,-1,35,6,3,0,15,1
2973,78,6,185,26,4,3,0,15,1
928,159,9,90,73,4,6,0,55,1
3543,2,10,61,73,5,4,0,16,1


In [5]:
# building the actual model

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(100, input_dim = x.shape[1], activation = 'relu',kernel_regularizer = l2(0.01)),
    tf.keras.layers.Dense(50, activation = 'relu',kernel_regularizer = l2(0.01)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(25, activation = 'relu',kernel_regularizer = l2(0.01)),
    tf.keras.layers.Dense(y.shape[1], activation='softmax')
])

In [6]:
# compiling the model

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto', restore_best_weights=True)

In [7]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=1000)


Train on 3839 samples, validate on 960 samples
Epoch 1/1000
3839/3839 - 2s - loss: 4.3323 - accuracy: 0.7153 - val_loss: 1.2375 - val_accuracy: 0.8406
Epoch 2/1000
3839/3839 - 0s - loss: 1.5655 - accuracy: 0.8054 - val_loss: 1.0105 - val_accuracy: 0.8156
Epoch 3/1000
3839/3839 - 0s - loss: 1.0280 - accuracy: 0.8278 - val_loss: 0.7953 - val_accuracy: 0.8813
Epoch 4/1000
3839/3839 - 0s - loss: 0.7920 - accuracy: 0.8497 - val_loss: 0.6375 - val_accuracy: 0.8906
Epoch 5/1000
3839/3839 - 0s - loss: 0.6909 - accuracy: 0.8575 - val_loss: 0.5614 - val_accuracy: 0.8865
Epoch 6/1000
3839/3839 - 0s - loss: 0.6909 - accuracy: 0.8549 - val_loss: 0.5361 - val_accuracy: 0.8750
Epoch 7/1000
3839/3839 - 0s - loss: 0.5921 - accuracy: 0.8580 - val_loss: 0.5091 - val_accuracy: 0.8698
Epoch 8/1000
3839/3839 - 0s - loss: 0.5769 - accuracy: 0.8593 - val_loss: 0.4840 - val_accuracy: 0.8844
Epoch 9/1000
3839/3839 - 1s - loss: 0.5428 - accuracy: 0.8692 - val_loss: 0.4688 - val_accuracy: 0.8917
Epoch 10/1000
383

In [8]:
# Checking for the model accuracy

from sklearn.metrics import accuracy_score
import numpy as np

pred = model.predict(x_test)
predict_classes = np.argmax(pred,axis=1)
expected_classes = np.argmax(y_test,axis=1)
correct = accuracy_score(expected_classes,predict_classes)
print(f"Accuracy: {correct}")

Accuracy: 0.8947916666666667


In [9]:
#Save the model
# serialize model to JSON
model_json = model.to_json()
with open("model_phishing.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_phishing.h5")
print("Saved model to disk")

Saved model to disk
